In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
from IPython.display import display
import scipy as sp

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = None   # To see the entire columns of dataframes
pd.options.display.max_rows = 100    


In [2]:
train_combat = pd.read_csv("data/train_combat.csv")
train_payment = pd.read_csv("data/train_payment.csv")
train_pledge = pd.read_csv("data/train_pledge.csv")
train_trade = pd.read_csv("data/train_trade.csv")
train_activity = pd.read_csv("data/train_activity.csv")
train_label = pd.read_csv("data/train_label.csv")

In [3]:
# Copy files in case of retrieving the original ones
combat = train_combat.copy()
pledge = train_pledge.copy()
payment = train_payment.copy()
trade = train_trade.copy()
activity = train_activity.copy()

In [4]:
# Total amount spent per account the whole period
train_label["total_spent"] = train_label["survival_time"] * train_label["amount_spent"]
label = train_label["acc_id"]

In [5]:
# No need to keep the rows whose "acc_id" does not appear in Label data
combat = combat[combat["acc_id"].isin(label)]
pledge = pledge[pledge["acc_id"].isin(label)]
payment = payment[payment["acc_id"].isin(label)]
trade = trade[trade["source_acc_id"].isin(label)]
trade = trade[trade["target_acc_id"].isin(label)]
activity = activity[activity["acc_id"].isin(label)]

## Common Features

In [6]:
# The number of characters per account
all_char = pd.concat([train_combat[["char_id", "acc_id"]], train_pledge[["char_id", "acc_id"]], train_activity[["char_id", "acc_id"]]])
char_count = all_char.groupby("char_id")["acc_id"].count()
char_count = pd.DataFrame({"char_count" : char_count})  # in case of merging

In [7]:
# Find all the unique servers
total_server = pd.concat([train_combat["server"], train_pledge["server"], train_trade["server"], train_activity["server"]])
server_list = total_server.unique()

# Encode servers into integer values
le = LabelEncoder()
le.fit(server_list)
for df in [combat, pledge, trade, activity]:
    df["server"] = le.transform(df["server"])

## Trade

In [8]:
trade.rename(columns={"source_acc_id" : "acc_id"}, inplace=True)

# Remove "source_char_idd" and "target_char_id"
trade.drop(["source_char_id", "target_char_id"], axis=1, inplace=True)

# Substitute NaNs with median at "item_price"
trade["item_price"].fillna(trade["item_price"].median(), inplace=True)

# Categorize transaction time
bins = [0, 60000, 120000, 180000, 239999]
bin_label = [0, 1, 2, 3]   
trade["time_bin"] = pd.cut(pd.to_numeric(trade["time"].str.replace(":", "")), bins=bins, labels=bin_label)
# Those who play during the time other than evening must be differentiated
trade["time_bin"] = trade["time_bin"].map({0:1, 1:1, 2:1, 3:0}) 
trade.drop("time", axis=1, inplace=True)

# Total price per each trade
trade["total_item_price"] = trade["item_amount"] * trade["item_price"]

# Make two trade DataFrames for each source and target
source_trade = trade.drop("target_acc_id", axis=1)
target_trade = trade.drop("acc_id", axis=1)
target_trade.rename(columns={"target_acc_id" : "acc_id"}, inplace=True)

# Total number of trade occurence per account the whole period
source_trade_count = source_trade["acc_id"].value_counts()
target_trade_count = target_trade["acc_id"].value_counts()

source_trade_count = pd.DataFrame({"acc_id" : source_trade_count.index,
                                   "count" : source_trade_count})    # Make a dataframe to merge based on "acc_id"
target_trade_count = pd.DataFrame({"acc_id" : target_trade_count.index,
                                   "count" : target_trade_count})

trade_count = pd.merge(source_trade_count, target_trade_count, on = "acc_id")
trade_count["count"] = trade_count["count_x"] + trade_count["count_y"]
trade_count.drop(["count_x", "count_y"], axis=1, inplace=True)

# Remove columns that do not seem necessary
trade.drop(["server", "target_acc_id", "item_type", "item_amount", "item_price"], axis=1, inplace=True)

trade = pd.concat(
    [pd.pivot_table(data=trade, index=["acc_id", "day"], values=["time_bin", "total_item_price"], aggfunc=sum), # sum all the prices
    pd.pivot_table(data=trade, index=["acc_id", "day"], values=["type"])], # mean for trade type
    axis=1)

## Pledge

In [9]:
pledge.columns

Index(['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

### 'play_char_cnt'
혈맹 내 유저의 접속률이 높을수록 생존에 영향이 있을거라 예상  
--> 접속률에 따른 점수 부여. 접속률 1위 혈맹에 1.0 , 나머지 혈맹이 1위 혈맹 대비 비례한 점수 부여.

In [ ]:
# 각 혈맹의 평균 유저 접속률 지표
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='play_char_cnt', aggfunc='mean')

# 정렬 후 1위 값으로 나눔 => 접속률 1위인 혈맹 1.0, 나머지 1.0보다 차례로 낮은 값.
avg_play_rate_per_pledge = pledge_pivot.play_char_cnt.sort_values(ascending=False)\
                                    / pledge_pivot.play_char_cnt.sort_values(ascending=False).iloc[0]

In [ ]:
to_be_merged = pd.DataFrame({'avg_play_rate_per_pledge' : avg_play_rate_per_pledge})
to_be_merged.head(10)

In [ ]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [ ]:
# 접속률 1위 혈맹 확인
pledge[pledge.pledge_id==17429]

### pledge_combat_cnt
마찬가지로 혈맹간 전투(공성 등) 같은 엔드 컨텐츠를 많이 즐기는 혈맹의 유저일 수록 생존률이 높을거라 예상  
--> 혈맹간 전투 횟수에 점수 부여. 위의 접속률과 같은 방식으로 점수 부여.

In [ ]:
# 혈맹간 총 전투 수 체크
pledge_pivot = pd.pivot_table(data=pledge, index=['pledge_id'], values='pledge_combat_cnt', aggfunc='sum')

total_combat_cnt_per_pledge = pledge_pivot.pledge_combat_cnt.sort_values(ascending=False)\
                                / pledge_pivot.pledge_combat_cnt.sort_values(ascending=False).iloc[0]

In [ ]:
to_be_merged = pd.DataFrame({'total_combat_cnt_per_pledge':total_combat_cnt_per_pledge})
to_be_merged.head(10)

In [ ]:
pledge = pd.merge(pledge, to_be_merged, on='pledge_id')

In [ ]:
# 혈맹간 전투 수 1위 혈맹 확인
pledge[pledge.pledge_id==36551]

In [ ]:
# The number of members in each pledge
pledge_num_people = pledge["pledge_id"].value_counts()

# Remove "char_id", "server", "pledge_id" (doesn't seem to matter)
pledge.drop(["char_id", "server", "pledge_id"], axis=1, inplace=True)

# Sum all the values according to days per account
pledge = pd.concat([pd.pivot_table(data=pledge.drop(['avg_play_rate_per_pledge','total_combat_cnt_per_pledge'], axis=1), index=["acc_id", "day"], aggfunc=sum),
                   pd.pivot_table(data=pledge, index=['acc_id','day'], values=['avg_play_rate_per_pledge','total_combat_cnt_per_pledge'])], axis=1)

# Change name of the columns which are same with the ones in combat file
rename_dict = {"etc_cnt" : "p_etc_cnt", "random_attacker_cnt" : "p_random_attacker_cnt",
              'same_pledge_cnt' : 'p_same_pledge_cnt', "temp_cnt" : "p_temp_cnt", 
               "random_defender_cnt" : "p_random_defender_cnt"}
pledge.rename(columns=rename_dict, inplace=True)

## Activity

In [ ]:
# Remove "char_id"
activity.drop("char_id", axis=1, inplace=True)

# Total play time per account the whole period
total_play = activity.groupby("acc_id")["playtime"].sum()
total_play = pd.DataFrame({"total_play_time" : total_play})  # in case of merging

# activity["cum_play_time"] = activity.groupby("acc_id")["playtime"].cumsum()

# Sum all the values per day for each account
activity = pd.concat([pd.pivot_table(data=activity.drop("server", axis=1), index=["acc_id", "day"], aggfunc=sum),
                     pd.pivot_table(data=activity, index=["acc_id", "day"], values=["server"])],
                    axis=1)

# Total exp
activity["total_exp"] = activity["solo_exp"] + activity["party_exp"] + activity["quest_exp"]

# How long do users spend time fishing?
activity["fishing_prop"] = activity["fishing"] / activity["playtime"]

In [ ]:
activity.columns

## Combat

In [ ]:
combat.columns

### class
"많은 사람들이 플레이 하는 주류 클래스 --> 성능이 좋은 클래스"  
"성능이 좋은 클래스를 하는 유저 --> 생존률 높지 않을까?"

|범주|직업|
|:---:|:---:|
|0|군주|
|1|기사|
|2|요정|
|3|마법사|
|4|다크엘프|
|5|용기사|
|6|환술사|
|7|전사|

In [ ]:
combat['class'] = combat['class'].astype('category')

In [ ]:
# 클래스별 전체 비율 조사
prop_class = combat['class'].value_counts() / combat['class'].value_counts().sum()
prop_class

In [ ]:
# 전체 누적합 60% 미만을 차지하는 주류 클래스 조사
class60 = prop_class[~(prop_class.cumsum()>0.6)].index # 기사, 요정, 마법사 (70%로 하면 전사까지.)

def isMajorClass(classs):
    if classs in class60:
        return 1
    else:
        return 0

combat['isMajorClass'] = combat['class'].apply(isMajorClass)
combat['isMajorClass'].head(10)

In [ ]:
# Remove "char_id", "class", "server"
combat.drop(["char_id", "class", "server"], axis=1, inplace=True)

# Max level? or Mean level?
max_level = combat.groupby("acc_id")["level"].max()
mean_level = combat.groupby("acc_id")["level"].mean()
combat.drop("level", axis=1, inplace=True)

# Sum all the values per day for each account
combat = pd.concat([pd.pivot_table(data=combat.drop('isMajorClass', axis=1), index=["acc_id", "day"], aggfunc=sum),
                   pd.pivot_table(data=combat, index=['acc_id', 'day'], values='isMajorClass', aggfunc='mean')], axis=1)
# isMajorClass mean 처리해서 주류 클래스만 플레이했을 경우 1.0, 그 외에는 비주류 클래스 캐릭터 수에 따라 평균값 떨어짐

In [ ]:
combat

## Payment

In [ ]:
# total_payment = payment.groupby("acc_id")["amount_spent"].sum()
# total_payment.head()

In [ ]:
train_label[train_label["acc_id"] == 8]

###### Why are they different????? ####

In [ ]:
# Check the results
display(combat.tail())
display(payment.head())
display(pledge.head())
display(trade.head())
display(activity.head())

In [ ]:
# Squeeze the whole dataframes into one
df = combat.join(pledge).join(trade).join(activity)

In [ ]:
df.isnull().sum()

In [ ]:
df[100:200]

In [ ]:
# '196818' 개의 NaN 값 생성 이유:
# 해당 유저가 '혈맹 전투 관련 활동'을 안한 날짜 존재.
# 평균으로 채워줄지, 0으로 채워줄지 고민
display(df.loc[38].total_combat_cnt_per_pledge)
display(pledge.loc[38].total_combat_cnt_per_pledge)

In [ ]:
to_be_dropped = ['play_char_cnt', 'pledge_combat_cnt']
to_be_filled0 = ['combat_char_cnt', 'combat_play_time', 'p_etc_cnt', 'non_combat_play_time', 
                 'p_random_attacker_cnt', 'p_random_defender_cnt', 'p_same_pledge_cnt', 
                 'p_temp_cnt', 'avg_play_rate_per_pledge', 'total_combat_cnt_per_pledge']
df = df.drop(to_be_dropped, axis=1) # 이 컬럼은 혈맹 단위로 대체됨
df[to_be_filled0] = df[to_be_filled0].fillna(0) # 이 컬럼은 혈맹 관련 활동을 안한 날 --> 0으로.

In [ ]:
df.isnull().sum()